In [1]:
from nltk.translate.meteor_score import meteor_score as calc_meteor
import nltk
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from datasets import load_dataset
import datasets
import torch
import pandas as pd

c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
models = [
    ["damapika/roberta-base_mod",386],
    ["damapika/distilbert-base-uncased_mod",384],
    ["damapika/electra-base-discriminator_squad_mod",386]
]
# Load the SQuAD validation dataset 
val_dataset = load_dataset("squad", split="validation")
results=[]

for model_name in models:
  tokenizer = AutoTokenizer.from_pretrained(model_name[0])
  model=AutoModelForQuestionAnswering.from_pretrained(model_name[0])
  result=[]
  meteor_scores=[]
  for example in val_dataset:
    
    # Get the reference answer and question
    reference_answer = example["answers"]["text"][0]
    question = example["question"]
    context = example["context"]

    # Tokenize the input
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt",max_length=model_name[1],truncation=True)

    # Generate an answer using the model
    outputs = model(**inputs)
    start_index = torch.argmax(outputs.start_logits)
    end_index = torch.argmax(outputs.end_logits)
    predicted_answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][start_index:end_index + 1].tolist()))
    # Compute METEOR score
    reference_tokens = nltk.word_tokenize(reference_answer.lower())
    predicted_tokens = nltk.word_tokenize(predicted_answer.lower())
    meteor_score = calc_meteor([reference_tokens], predicted_tokens)
    meteor_scores.append(meteor_score)
    print(meteor_score)

    # Calculate the average scores
  avg_meteor_score = sum(meteor_scores) / len(meteor_scores)
  result.append(model_name[0])
  result.append(avg_meteor_score)
  results.append(result)
  # Print the average evaluation results
  print(model_name[0])
  print(f"Average meteor score: {avg_meteor_score}")
df = pd.DataFrame(results)
df.to_csv('qa_models_squad_meteor_eval.csv') 

Found cached dataset squad (C:/Users/dama_/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


0.9375


TypeError: 'float' object is not callable